Much of this code inspired by https://github.com/ageron/handson-ml/blob/master/02_end_to_end_machine_learning_project.ipynb from the book "Hands-On Machine Learning with Scikit-Learn & TensorFlow" by Aurélien Geron.

Setup defaults and import libraries.

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

# to make this notebook's output stable across runs
np.random.seed(17)

Load the data

In [2]:
coffee_data = pd.read_csv("datasets//arabica_data_cleaned.csv", index_col=[0])
coffee_data = coffee_data.dropna(subset=["Country.of.Origin","Variety","Processing.Method"])

Create test and training datasets

In [3]:
from sklearn.model_selection import train_test_split

train_set, test_set =  train_test_split(coffee_data, test_size=0.2, random_state=17)

coffee = train_set.drop("Total.Cup.Points",axis=1)
coffee_labels = train_set["Total.Cup.Points"].copy()

Write all of our data transformation steps

In [4]:
from sklearn.preprocessing import FunctionTransformer

def null_altitude_outliers(X):
    X[X < 200] = None
    X[X > 5000] = None
    return X

Build our pipeline

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

# We need these because if a value doesn't appear in the training set, it will fail in the test set
unique_countries = list(coffee_data["Country.of.Origin"].unique())
unique_varieties = list(coffee_data["Variety"].unique())

pipeline = ColumnTransformer([
        ('country_encoding', Pipeline([
            ('encoder', OrdinalEncoder(categories=[unique_countries]))
        ]),["Country.of.Origin"]),
        ('variety_encoding', Pipeline([
            ('encoder', OrdinalEncoder(categories=[unique_varieties]))
        ]),["Variety"]),
        ('processing_encoding', Pipeline([
            ('encoder', OrdinalEncoder())
        ]),["Processing.Method"]),
        ('numerical_data', Pipeline([
            ('null_outliers', FunctionTransformer(null_altitude_outliers,validate=False)),
            ('imputer', SimpleImputer(strategy="median")),
            ('std_scaler', MinMaxScaler())
        ]),["altitude_mean_meters"])
    ])

coffee_prepared = pipeline.fit_transform(coffee)

Train on the prepared data.  Test various algorithms and their errors.

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linear_regression = LinearRegression()
linear_regression.fit(coffee_prepared, coffee_labels)

coffee_predictions = linear_regression.predict(coffee_prepared)
linear_mse = mean_squared_error(coffee_labels,coffee_predictions)
linear_rmse = np.sqrt(linear_mse)
linear_rmse

2.609355162708351

In [7]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(coffee_prepared, coffee_labels)

coffee_predictions = tree_reg.predict(coffee_prepared)
tree_mse = mean_squared_error(coffee_labels,coffee_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

1.547140261435914

In [8]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(coffee_prepared, coffee_labels)

coffee_predictions = forest_reg.predict(coffee_prepared)
forest_mse = mean_squared_error(coffee_labels,coffee_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

1.6830327124524418

In [9]:
from sklearn.model_selection import cross_val_score

def display_scores(name,scores):
    print("Estimator Name:", name)
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())
    print("")

scores = cross_val_score(linear_regression, coffee_prepared, coffee_labels, scoring="neg_mean_squared_error", cv=10)
linear_rmse_scores = np.sqrt(-scores)
display_scores("linear",linear_rmse_scores)

scores = cross_val_score(tree_reg, coffee_prepared, coffee_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
display_scores("tree",tree_rmse_scores)

scores = cross_val_score(forest_reg, coffee_prepared, coffee_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)
display_scores("forest",forest_rmse_scores)

Estimator Name: linear
Scores: [3.02849249 2.96519542 2.25563438 2.25502797 2.56039309 2.10010873
 3.42839173 2.74842951 2.34846564 2.23623238]
Mean: 2.59263713373608
Standard deviation: 0.4134423003353246

Estimator Name: tree
Scores: [3.77521803 2.82958078 2.37696262 2.4319268  3.68829751 2.58867205
 3.66951236 2.6078661  2.44266597 2.52819137]
Mean: 2.893889359118404
Standard deviation: 0.5483136241345696

Estimator Name: forest
Scores: [3.20400837 2.76156558 2.08012415 2.07747684 2.78563371 2.41456572
 3.30699095 2.56735791 2.33476961 2.23463517]
Mean: 2.5767128019751118
Standard deviation: 0.41260078019548674



Perform a grid search for hyper parameter tuning

In [10]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [100,150,180,200,250,300,350,400], 'max_features': [1,2,3,4]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [100,150,180,200,250,300,350,400], 'max_features': [1,2,3,4]},
  ]

forest_reg = RandomForestRegressor(random_state=17)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(coffee_prepared, coffee_labels)

grid_search.best_params_

{'max_features': 1, 'n_estimators': 250}

In [11]:
feature_importances = grid_search.best_estimator_.feature_importances_
#country,variety,processing,altitude
feature_importances

array([0.25412312, 0.189736  , 0.05873432, 0.49740656])

The above feature importance indicates that altitude has the largest importance, followed by country of origin, variety, and finally processing method.

In [12]:
final_model = grid_search.best_estimator_

X_test = test_set.drop("Total.Cup.Points",axis=1)
y_test = test_set["Total.Cup.Points"].copy()

X_test_prepared = pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

2.6202736927843424

In [13]:
estimator = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=4, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=250, n_jobs=None, oob_score=False,
                      random_state=17, verbose=0, warm_start=False)

full_pipeline_with_predictor = Pipeline([
        ("preparation", pipeline),
        ("estimator", estimator)
    ])

full_pipeline_with_predictor.fit(coffee, coffee_labels)

data = coffee.copy().iloc[:1]
data["Country.of.Origin"] = "Kenya"
data["Variety"] = "Bourbon"
data["Processing.Method"] = "Natural / Dry"
data["altitude_mean_meters"] = 2000

result = full_pipeline_with_predictor.predict(data)
print(result)

[84.23760324]


Save the model to a pickle file

In [14]:
import joblib
joblib.dump(full_pipeline_with_predictor, "coffee_model.pkl")

['coffee_model.pkl']